In [4]:
import tensorflow as tf
import os
from tensorflow.keras import Model

## Data Path

In [ ]:
import cv2
import os

# Fungsi untuk me-resize gambar
def resize_image(input_folder, output_folder, size=(300, 300)):
    # Membuat folder output jika belum ada
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop melalui semua file dalam folder
    for filename in os.listdir(input_folder):
        # Cek apakah file adalah gambar (berdasarkan ekstensi file)
        if filename.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.tiff')):
            # Baca gambar
            img_path = os.path.join(input_folder, filename)
            image = cv2.imread(img_path)
            
            # Jika gambar berhasil dibaca
            if image is not None:
                # Resize gambar
                resized_image = cv2.resize(image, size)
                
                # Simpan gambar yang sudah di-resize ke folder output
                output_path = os.path.join(output_folder, filename)
                cv2.imwrite(output_path, resized_image)
                
                print(f'{filename} berhasil di-resize dan disimpan di {output_folder}')
            else:
                print(f'Gagal membaca {filename}')
        else:
            print(f'{filename} bukan file gambar')

# Tentukan folder input dan output
input_folder = r'data\test\sehat'

output_folder = r'data\test\t'

# Panggil fungsi untuk resize semua gambar
resize_image(input_folder, output_folder)

In [ ]:
basedir = 'data'
print(os.listdir(basedir))

In [6]:
traindir = os.path.join(basedir, 'train')
valdir = os.path.join(basedir, 'val')
testdir = os.path.join(basedir, 'test')

## Data Preprocess

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(traindir,
                                                            shuffle=True,
                                                            batch_size=2,
                                                            image_size=(224, 224))
valdir_dataset = tf.keras.utils.image_dataset_from_directory(valdir,
                                                            shuffle=True,
                                                            batch_size=2
                                                             ,
                                                            image_size=(224, 224))

                                                  

train_dataset_final = train_dataset.cache().shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE)
valdir_dataset_final = valdir_dataset.cache().shuffle(buffer_size=1000).prefetch(buffer_size=tf.data.AUTOTUNE)


In [19]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.2),
  tf.keras.layers.RandomHeight(0.2),
  tf.keras.layers.RandomWidth(0.2),
], name="data_augmentation")

In [20]:
traain_dataset_final = train_dataset_final.map(lambda x, y: (data_augmentation(x, training=True), y))

## Callback

In [61]:

class CustomEarlyStopping(tf.keras.callbacks.Callback):
    def __init__(self, patience=0):
        super(CustomEarlyStopping, self).__init__()
        self.patience = patience
        self.best_val_accuracy = 0
        self.best_val_loss = float('inf')
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        current_val_accuracy = logs.get('val_accuracy')
        current_val_loss = logs.get('val_loss')

        if current_val_accuracy > self.best_val_accuracy and current_val_loss < self.best_val_loss:
            self.best_val_accuracy = current_val_accuracy
            self.best_val_loss = current_val_loss
            self.wait = 0  # Reset wait if we have a new best
        else:
            self.wait += 1

        if self.wait >= self.patience:
            print(f"\nEarly stopping at epoch {epoch + 1}")
            self.model.stop_training = True


##Buat Model

In [8]:
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, BatchNormalization
from tensorflow.keras.applications import MobileNetV2


In [63]:
# Hyperparameters
patience = 25  # Number of epochs with no improvement after which training will be stopped

# Instantiate your custom callback
custom_callback = CustomEarlyStopping(patience=patience)

In [12]:

pretrained_model = tf.keras.applications.MobileNetV3Large(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    
    pooling='avg'
)



In [37]:
from tensorflow.keras.layers import Dense, Dropout, Concatenate
from tensorflow.keras.models import Model
import tensorflow as tf

# Membekukan pretrained model
pretrained_model.trainable = False

# Input dari pretrained model
inputs = pretrained_model.input

# Menggunakan output dari pretrained model
x1 = Dense(128, activation=tf.nn.relu)(pretrained_model.output)
x2 = Dense(128, activation=tf.nn.relu)(pretrained_model.output)
x3 = Dense(128, activation=tf.nn.relu)(pretrained_model.output)

# Menggabungkan output menggunakan Concatenate
merge = Concatenate()([x1, x2, x3])

# Melanjutkan dengan Dense layers
x = Dense(128, activation=tf.nn.relu)(merge)
x = Dense(64, activation=tf.nn.relu)(x)
x = Dropout(0.2)(x)
outputs = Dense(6, activation=tf.nn.softmax)(x)

# Membuat model
model = Model(inputs=inputs, outputs=outputs)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Summary
model.summary()


Model: "functional_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ rescaling_1 (Rescaling)       │ (None, 224, 224, 3)       │               0 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv (Conv2D)                 │ (None, 112, 112, 16)      │             432 │ rescaling_1[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv_bn (BatchNormalization)  │ (None, 112, 112, 16)      │              64 │ conv[0][0]                 │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation_20 (Activation)    │ (None, 112, 112, 16)      │               0 │ conv_bn[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise       │ (None, 112, 112, 16)      │             144 │ activation_20[0][0]        │
│ (DepthwiseConv2D)             │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_depthwise_bn    │ (None, 112, 112, 16)      │              64 │ expanded_conv_depthwise[0… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_19 (ReLU)               │ (None, 112, 112, 16)      │               0 │ expanded_conv_depthwise_b… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project         │ (None, 112, 112, 16)      │             256 │ re_lu_19[0][0]             │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_project_bn      │ (None, 112, 112, 16)      │              64 │ expanded_conv_project[0][… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_add (Add)       │ (None, 112, 112, 16)      │               0 │ activation_20[0][0],       │
│                               │                           │                 │ expanded_conv_project_bn[… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_1_expand        │ (None, 112, 112, 64)      │           1,024 │ expanded_conv_add[0][0]    │
│ (Conv2D)                      │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ expanded_conv_1_expand_bn     │ (None, 112, 112, 64)      │             256 │ expanded_conv_1_expand[0]… │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ re_lu_20 (ReLU)               │ (None, 112, 112, 64)      │               

 Total params: 3,423,302 (13.06 MB)

 Trainable params: 426,950 (1.63 MB)

 Non-trainable params: 2,996,352 (11.43 MB)

In [ ]:
history = model.fit(
    train_dataset_final,

    validation_data=valdir_dataset_final,

    epochs=100,

)

In [ ]:
model.summary()

In [30]:
model.save('kucingku_sehat_adamv1.keras')

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt  # Untuk menampilkan gambar

# Load model h5
model = tf.keras.models.load_model('kucingku_sehat.h5')

# Load the image
image_path = 'acne.jpg'
image = tf.keras.utils.load_img(image_path, target_size=(224, 224))  # Resize langsung saat load

# Convert the resized image to a numpy array
img_array = tf.keras.utils.img_to_array(image)

# Normalisasi gambar (pastikan ini sesuai dengan preprocessing model Anda)
img_array = img_array / 255.0  # Normalisasi jika model dilatih dengan nilai [0, 1]

# Expand dimensions to match the model input shape (1, 224, 224, 3)
img_bat = tf.expand_dims(img_array, axis=0)

# Menampilkan gambar
plt.imshow(image)
plt.axis('off')  # Menghilangkan sumbu
plt.show()

# Make predictions
predict = model.predict(img_bat)
score = tf.nn.softmax(predict[0])  # Ambil elemen pertama untuk skor

# Data kategori (update sesuai label Anda)
data_cat = ['acne', 'dermatitis', 'eosinophilicplaque', 'kutu', 'rodentulcer', 'sehat']

# Tampilkan prediksi
predicted_label = data_cat[np.argmax(score)]
confidence = 100 * np.max(score)
print(f'Kondisi: {predicted_label} ({confidence:.2f}% confidence)')


In [ ]:
from tensorflow.keras.models import load_model

# Memuat model
model = load_model('kucingku_sehat_real.h5')

# Sekarang model siap digunakan
